# Modelling NIQS Hardware TF

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import torch.optim as optim
import tensorflow as tf
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *

KeyboardInterrupt: 

In [10]:
n = 3
d = 2**n
input_list, counts_list = pickle.load(open("..\..\data\threeQubits_expectation_POVM.p", "rb"))
counts_error = counts_list[-d**2:]
corr_mat = sum([generate_corruption_matrix(counts_error[d*i:d*(i+1)]) for i in range(d)])/d
povm = corr_mat_to_povm(corr_mat)
print(corr_mat)
print(sum(povm))

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\data\threeQubits_expectation_POVM.p'

In [2]:
n = 2
d = 2**n
rank = 4
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

kraus_target = KrausMap(
#                 U = None,
#                 c = None,
                 d = d,
                 rank = rank,
                 trainable = False,
                 )

In [3]:
N = 540
state_index, observ_index = index_generator(n, N, trace=True)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):

    config = numberToBase(i, 6, n)
    state = tf.cast(prepare_input(config), dtype=tf.complex64)

    config = numberToBase(j, 4, n)
    U_basis, observable  = pauli_observable(config, return_mode = "unitary")
    U_basis, observable = tf.cast(U_basis, dtype = tf.complex64), tf.cast(observable, dtype = tf.complex64)
    
    input_list.append([state, U_basis, observable])
    
target_list = [measurement(kraus_target.apply_map(input[0]), input[1]) for input in input_list]
input_train, input_test = input_list[:440], input_list[440:]
target_train, target_test = target_list[:440], target_list[440:] 
optimizer = tf.optimizers.Adam(learning_rate=0.025)

In [4]:
povm = corr_mat_to_povm(np.eye(d))
kraus_model = KrausMap(
#                 U = None,
#                 c = None,
                 d = d,
                 rank = rank,
                 trainable = True,
                 povm = povm,
                 )

model = ModelQuantumMap(q_map = kraus_model,
                        loss = probs_loss,
                        input_list = input_list,
                        target_list = target_list,
                        input_val_list = input_list,
                        target_val_list = target_list,
                        optimizer = optimizer,
                        )

In [5]:
model.train(num_iter = 1000, N = 10)

  0%|          | 0/1000 [00:00<?, ?it/s]

0 (0.09833557+1.402405e-09j)
1 (0.0976313+1.5992718e-09j)
2 (0.09639015+1.7586755e-09j)
3 (0.095115125+5.8166083e-10j)
4 (0.09361214+1.2002336e-09j)
5 (0.09213511+4.1117594e-09j)
6 (0.090740606+1.969743e-09j)
7 (0.089554116+6.897863e-10j)
8 (0.08837625+1.3479738e-09j)
9 (0.086716086+1.5902356e-09j)
10 (0.08515311+1.8526803e-09j)
11 (0.08359885+2.433932e-09j)
12 (0.082134284+1.9260133e-09j)
13 (0.08076902+1.580483e-09j)
14 (0.07957444+1.4585881e-09j)
15 (0.07815617+2.013101e-09j)
16 (0.07655589+1.2058968e-09j)
17 (0.074892946+1.8523847e-09j)
18 (0.073151805+2.033778e-09j)
19 (0.07160288+1.1252221e-09j)
20 (0.069981806-1.166961e-10j)
21 (0.06833158+3.110462e-09j)
22 (0.06673961+1.6500933e-09j)
23 (0.06520693+2.178843e-09j)
24 (0.06389063+1.2914921e-09j)
25 (0.06262157+2.1997706e-09j)
26 (0.061473083+7.910288e-10j)
27 (0.06038545+2.5130695e-09j)
28 (0.05914773+1.1842195e-10j)
29 (0.058004282+1.657903e-09j)
30 (0.056990054+1.4755499e-09j)
31 (0.055788703+1.5828194e-10j)
32 (0.054513264+2.3

254 (0.003418149+5.9995564e-11j)
255 (0.0034156838+4.7954783e-11j)
256 (0.0034073624-3.7978783e-11j)
257 (0.0033957013+2.5010044e-11j)
258 (0.0033771503-4.9419794e-13j)
259 (0.0033602107+1.835316e-12j)
260 (0.003340448-7.575039e-11j)
261 (0.0033255583-1.7481263e-11j)
262 (0.003307292+1.11119634e-10j)
263 (0.0032861396+4.5849453e-11j)
264 (0.0032712538+3.8914157e-11j)
265 (0.0032454133+8.386482e-13j)
266 (0.0032251712-2.2661528e-12j)
267 (0.0032073946-3.9416127e-11j)
268 (0.003191449+8.176989e-12j)
269 (0.0031826815-2.8335468e-11j)
270 (0.0031714651-1.2076032e-10j)
271 (0.0031547416+1.0174619e-11j)
272 (0.0031364132+3.6784905e-11j)
273 (0.0031175704-1.276633e-12j)
274 (0.003095943-8.298194e-11j)
275 (0.0030715547-5.220308e-11j)
276 (0.0030480865-6.5830376e-11j)
277 (0.0030229727+2.4397814e-11j)
278 (0.0030006212+9.521629e-12j)
279 (0.002973261-1.0762964e-10j)
280 (0.0029583168+1.1472952e-10j)
281 (0.0029497924+3.115864e-11j)
282 (0.0029389819+2.4390646e-11j)
283 (0.0029282698-9.271609e-

KeyboardInterrupt: 

In [ ]:
def apply_map(state, kraus_list):
    state = [K@state@tf.linalg.adjoint(K) for K in kraus_list]
    state = tf.math.reduce_sum(tf.stack(state), axis=0)
    return state


def expectation_value(state, observable):
    ev = tf.linalg.trace(observable@state)
    return ev

In [ ]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 8

A = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
B = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
G = A + 1j*B
Q, R = tf.linalg.qr(G, full_matrices = False)
D = tf.linalg.tensor_diag_part(R)
D = tf.math.sign(D)
D = tf.linalg.diag(D)
U = Q@D


kraus_target_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]

In [ ]:
N = 100
state_index, observ_index = index_generator(n, N, trace=False)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):

    config = numberToBase(i, 6, n)
    state = tf.cast(prepare_input(config), dtype=tf.complex64)

    config = numberToBase(j, 3, n)
    observable = tf.cast(pauli_observable(config), dtype=tf.complex64)
    
    input_list.append([state, observable])

target_list = [expectation_value(apply_map(input[0], kraus_target_list), input[1]) for input in input_list]
optimizer = tf.optimizers.Adam(learning_rate=0.05)

In [ ]:
A = tf.Variable(tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64), trainable=True)
B = tf.Variable(tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64), trainable=True)

for i in range(1000):
    with tf.GradientTape() as tape:
        G = A + 1j*B
        Q, R = tf.linalg.qr(G, full_matrices = False)
        D = tf.linalg.tensor_diag_part(R)
        D = tf.math.sign(D)
        D = tf.linalg.diag(D)
        U = Q@D

        kraus_model_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]
        pred_list = [expectation_value(apply_map(input[0], kraus_model_list), input[1]) for input in input_list]
        loss = tf.math.reduce_mean(tf.stack([(target - predicted)**2 for target, predicted in zip(target_list, pred_list)]))

    grads = tape.gradient(loss, [A, B])
    optimizer.apply_gradients(zip(grads, [A, B]))
    
    print(f"loss: {loss: .4f}, {tf.math.reduce_mean(tf.math.abs(G))}")

In [ ]:
np.random.seed(43)
A = np.random.normal(0,1, (2,2))
B = np.random.normal(0,1, (2,2))
C = np.random.normal(0,1, (2,2))

A = A@A.T
B = B@B.T
C = C@C.T

D = A + B + C
d = np.linalg.inv(sqrtm(D))
A = d@A@d
B = d@B@d
C = d@C@d

In [ ]:
print(A)
print(B)
print(C)
print(A + B + C)
print(np.linalg.eig(A)[0])
print(np.linalg.eig(B)[0])
print(np.linalg.eig(C)[0])